In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#define function that takes a list of hyperlinks and returns

In [2]:
URL = "https://liftblog.com/united-states/"
page = requests.get(URL)

#create a beautifulsoup object with the page content and choose the appropriate parser
soup = BeautifulSoup(page.content, "html.parser")

# grabs container for job postings
results = soup.find(id="post-1491")

#there are two lists on this page.  first is the states and second is sharing links
states = results.find_all("ul")[0].find_all("a")


for state in states:
    state_name = state.text.strip()
    state_link = state['href']
    print(state_name)
    print(state_link)



Alabama
https://liftblog.com/alabama/
Alaska
https://liftblog.com/alaska/
Arizona
https://liftblog.com/arizona/
California
https://liftblog.com/california/
Colorado
https://liftblog.com/colorado/
Connecticut
https://liftblog.com/connecticut/
Florida
https://liftblog.com/florida/
Georgia
https://liftblog.com/georgia/
Idaho
https://liftblog.com/idaho/
Illinois
https://liftblog.com/illinois/
Indiana
https://liftblog.com/indiana/
Iowa
https://liftblog.com/iowa/
Kansas
https://liftblog.com/kansas/
Kentucky
https://liftblog.com/kentucky/
Maine
https://liftblog.com/maine/
Maryland
https://liftblog.com/wisp-md/
Massachusetts
https://liftblog.com/massachusetts/
Michigan
https://liftblog.com/michigan/
Minnesota
https://liftblog.com/minnesota/
Mississippi
https://liftblog.com/mississippi/
Missouri
https://liftblog.com/missouri/
Montana
https://liftblog.com/montana/
Nebraska
https://liftblog.com/nebraska/
Nevada
https://liftblog.com/nevada/
New Hampshire
https://liftblog.com/new-hampshire
New Jers

In [4]:
URL = "https://liftblog.com/alabama/"
page = requests.get(URL)

#create a beautifulsoup object with the page content and choose the appropriate parser
soup = BeautifulSoup(page.content, "html.parser")

# grabs container for list of resorts
results = soup.find("div", class_="entry-content")

ski_resorts = results.find_all("li")[0]

for resort in ski_resorts:
    resort_name = resort.text.strip()
    resort_link = resort['href']
    print(resort_name)
    print(resort_link)

Montgomery Zoo
https://liftblog.com/montgomery-zoo-al/


In [32]:
URL = "https://liftblog.com/montgomery-zoo-al/"
page = requests.get(URL)

#create a beautifulsoup object with the page content and choose the appropriate parser
soup = BeautifulSoup(page.content, "html.parser")

results = soup.find("div", class_="entry-content")

#get the link to the google sheet
google_sheet = results.find("a")['href']

google_page = requests.get(URL)
google_soup = BeautifulSoup(google_page.content, "html.parser")

google_table = google_soup.find_all("table")

print(google_table)


[]


In [39]:
sheet_id = "1XqOtPkiE_Q0dfGSoyxrH730RkwrTczcRbDeJJpqRByQ"
sheet_name = "sample_1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df = pd.read_csv(url)
print(df.head())

  Name  Attribute 1  Attribute 2  Attribute 3 label
0  ABC           97         7.98            2     A
1  DEF           34         5.36            3     B
2  GHI           65         4.31            4     B
3  JKL           57         3.47            7     A
4  MNO           32         9.24            5     A
